In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.special import wofz

# Physical constants
c = 2.998e5  # Speed of light in km/s
c_cm_s = 2.998e10  # Speed of light in cm/s
k_B = 1.38e-23  # Boltzmann constant in J/K
h = 6.626e-34  # Planck's constant in J*s

def voigt_profile(nu, nu_0, A21, T, m, v_shift):
    """Compute the Voigt profile given the input parameters."""
    # Doppler broadening width
    delta_nu_D = (nu_0 / c) * np.sqrt(2 * k_B * T / m)

    # Lorentzian width (natural broadening)
    gamma_L = A21 / (4 * np.pi)

    # Voigt profile parameters
    x = (nu - nu_0 - v_shift * nu_0 / c) / delta_nu_D
    a = gamma_L / delta_nu_D
    
    # Compute the Voigt profile using Faddeeva function
    phi_V = np.real(wofz(x + 1j * a)) / (delta_nu_D * np.sqrt(np.pi))
    
    return phi_V

def plot_profiles(lambda_0=1216, unit="Angstrom", A21=6e8, T=10000, m=1.67e-24, v_shift=0, L=1e14, N1=1.0):
    """Plots the absorption line profile and intensity I = exp(-tau)."""

    # Convert wavelength (Angstrom) to frequency (Hz)
    nu_0 = c_cm_s / (lambda_0 * 1e-8)  # Convert Angstrom to cm then to Hz
    
    # Frequency range
    nu = np.linspace(nu_0 * 0.99, nu_0 *1.01, 500)
    
    # Compute Voigt profile
    phi_V = voigt_profile(nu, nu_0, A21, T, m, v_shift)
    
    # Compute absorption coefficient
    B12 = (c**3 / (8 * np.pi * h * nu_0**3)) * (A21 * 2)  # Assume g2/g1 = 2 for simplicity
    alpha_nu = (h * nu_0 / (4 * np.pi)) * B12 * N1 * phi_V

    # Optical depth and intensity
    tau = L * alpha_nu
    I = np.exp(-tau)

    # Create figure with two panels
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # Convert to wavelength if needed
    if unit == "Angstrom":
        wavelength = c_cm_s / nu * 1e8  # Convert Hz to Angstrom
        xlabel = "Wavelength (Å)"
    else:
        wavelength = nu / 1e12  # Convert Hz to THz
        xlabel = "Frequency (THz)"

    # Plot Line Profile
    ax[0].plot(wavelength, phi_V, label="Voigt Profile", color='b')
    ax[0].set_xlabel(xlabel)
    ax[0].set_ylabel(r"$\phi_V(\nu)$")
    ax[0].set_title("Absorption Line Profile")
    ax[0].legend()
    ax[0].grid()

    # Plot Intensity
    ax[1].plot(wavelength, I, label=r"$I = e^{-\tau}$", color='r')
    ax[1].set_xlabel(xlabel)
    ax[1].set_ylabel("Intensity")
    ax[1].set_title("Intensity Profile")
    ax[1].legend()
    ax[1].grid()

    plt.tight_layout()
    plt.show()

# Create interactive widgets
interact(plot_profiles, 
         lambda_0=widgets.FloatLogSlider(value=1216, base=10, min=2, max=6, step=0.0001, description="λ₀ (Å)"),
         unit=widgets.RadioButtons(options=["THz", "Angstrom"], value="Angstrom", description="X-Axis"),
         A21=widgets.FloatLogSlider(value=6e8, base=10, min=-15, max=10, step=0.1, description="A₂₁ (s⁻¹)"),
         T=widgets.FloatLogSlider(value=10000, base=10, min=2, max=5, step=0.1, description="T (K)"),
         m=widgets.FloatLogSlider(value=1.67e-24, base=10, min=-28, max=-23, step=0.1, description="Mass (g)"),
         v_shift=widgets.FloatSlider(value=0, min=-1000, max=1000, step=1, description="Velocity (km/s)"),
         L=widgets.FloatLogSlider(value=1e14, base=10, min=12, max=22, step=0.1, description="L (cm)"),
         N1=widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=10, step=0.1, description="N₁ (cm⁻³)"));

interactive(children=(FloatLogSlider(value=1216.0, description='λ₀ (Å)', max=6.0, min=2.0, step=0.0001), Radio…